In [1]:
import torch
from eval_quantize import Classifier

/home/yuhsinchan/anaconda3/envs/general_slu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_fp32 = Classifier()
model_fp32.eval()

model_fp32.qconfig = torch.quantization.get_default_qconfig("fbgemm")
model_fp32_fused = torch.quantization.fuse_modules(
    model_fp32, [["conv", "batchnorm"]]
)
model_fp32_prepared = torch.quantization.prepare(model_fp32_fused)

/home/yuhsinchan/anaconda3/envs/general_slu/lib/python3.10/site-packages/torch/ao/quantization/observer.py:176: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [4]:
model_fp32_prepared.load_state_dict(torch.load("model_quantize.pth"))

model_int8 = torch.quantization.convert(model_fp32_prepared)

In [5]:
model_int8

Classifier(
  (quant): Quantize(scale=tensor([0.0673]), zero_point=tensor([58]), dtype=torch.quint8)
  (conv): QuantizedConv1d(8, 10, kernel_size=(3,), stride=(1,), scale=5.970702171325684, zero_point=72)
  (batchnorm): Identity()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): QuantizedLinear(in_features=30, out_features=27, scale=1740.132568359375, zero_point=45, qscheme=torch.per_channel_affine)
  (dequant): DeQuantize()
)

In [6]:
model_int8.state_dict()

OrderedDict([('quant.scale', tensor([0.0673])),
             ('quant.zero_point', tensor([58])),
             ('conv.weight',
              tensor([[[ 29.8405,  15.0377,  27.4908],
                       [  4.9343,   0.4699,   8.2238],
                       [-26.0811,   9.8685,  -9.1636],
                       [ 11.0433,   3.2895,   5.6391],
                       [  6.1091,   1.6448,   0.7049],
                       [-20.2069, -15.5077, -13.3930],
                       [ 15.9776,  15.5077,  13.8629],
                       [ -9.6335,  -7.5189,  -9.1636]],
              
                      [[ 14.2402,  13.5829,  10.2967],
                       [ 13.8020, -16.8691,  -0.2191],
                       [ 27.8231,  -1.5336,  19.9362],
                       [-12.4875,  -0.8763, -10.2967],
                       [ -9.8586,  -0.4382,   1.5336],
                       [-13.3638,  -7.6678,  -9.2013],
                       [-16.8691, -15.3356, -20.5935],
                       [ 10.5158,

In [8]:
model_fp32_prepared

Classifier(
  (quant): QuantStub(
    (activation_post_process): HistogramObserver()
  )
  (conv): Conv1d(
    8, 10, kernel_size=(3,), stride=(1,)
    (activation_post_process): HistogramObserver()
  )
  (batchnorm): Identity()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(
    in_features=30, out_features=27, bias=True
    (activation_post_process): HistogramObserver()
  )
  (dequant): DeQuantStub()
)

In [14]:
model_int8.state_dict()["conv.weight"][0]

tensor([[ 29.8405,  15.0377,  27.4908],
        [  4.9343,   0.4699,   8.2238],
        [-26.0811,   9.8685,  -9.1636],
        [ 11.0433,   3.2895,   5.6391],
        [  6.1091,   1.6448,   0.7049],
        [-20.2069, -15.5077, -13.3930],
        [ 15.9776,  15.5077,  13.8629],
        [ -9.6335,  -7.5189,  -9.1636]], size=(8, 3), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.23496443033218384,
       zero_point=0)